In [ ]:
import os
import natsort
import pandas as pd
import numpy
import math
import re

class trajectory: 
############################################################################################
############################################################################################
############################################################################################
    def __init__(self):
        ### SImulation step, Number of particles, Number of molecules
        self.step=0
        self.NumPart=0
        self.NumMole=0
        ### Box
        self.boxX=[0.0,0.0]
        self.boxY=[0.0,0.0]
        self.boxZ=[0.0,0.0]
        self.Lx=0.0
        self.Ly=0.0
        self.Lz=0.0
        ### Trajectory
        self.traj = pd.DataFrame({})
        ### For the linked cell list
        self.Ncells=0       ### Number of cells in linked cell list
        self.lscl=[]
        self.head=[]
        
############################################################################################
###### Read frame in lammps format
############################################################################################        
    def read_frame(self,file):
                    
        f=open(file)

        ### Read head of the file
        f.readline()
        
        line = f.readline()
        partL = line.split()
        self.step=int(partL[0])
        
        f.readline()
        
        line = f.readline()
        partL = line.split()
        self.NumPart=int(partL[0])
        
        f.readline()
        
        line = f.readline()
        partL = line.split()
        self.boxX=[float(partL[0]),float(partL[1])]
        self.Lx=float(partL[1])-float(partL[0])
        
        line = f.readline()
        partL = line.split()
        self.boxY=[float(partL[0]),float(partL[1])]
        self.Ly=float(partL[1])-float(partL[0])
        
        line = f.readline()
        partL = line.split()
        self.boxZ=[float(partL[0]),float(partL[1])]
        self.Lz=float(partL[1])-float(partL[0])
        
        line=f.readline()
        fields=line.split()[2:]
            
        self.traj=pd.read_csv(file, sep=" ", skiprows = 9,header = None)
        self.traj.columns =fields
                

#######################################################################
######### Recorre la lista de datos, obteniendo el número de partículas atrapadas en el anillo y en la circunferencia interior
#######################################################################
path='E:\\1_TFG\\TFG\\Simulaciones\\Tartarus-Punto\\'

Est_estacionario=449

R1_folders=os.listdir(path)
for R1 in R1_folders:
    R1_path=path+R1+'\\'
    gamma_folders=os.listdir(R1_path)
    for gamma in gamma_folders:
        gamma_path=R1_path+gamma+'\\'
        aa_folders=os.listdir(gamma_path)
        for aa in aa_folders:
            aa_path=gamma_path+aa+'\\'
            bb_folders=os.listdir(aa_path)
            for bb in bb_folders:
                bb_path=aa_path+bb+'\\'
                delta_folders=os.listdir(bb_path)
                for delta in delta_folders:
                    delta_path=bb_path+delta+'\\'
                    L1_folders=os.listdir(delta_path)
                    for L1 in L1_folders:
                        L1_path=delta_path+L1+'\\'
                        Nf_folders=os.listdir(L1_path)
                        for Nf in Nf_folders:
                            Nf_path=L1_path+Nf+'\\'
                            lp_folders=os.listdir(Nf_path)
                            for lp in lp_folders:
                                lp_path=Nf_path+lp+'\\'
                                v_folders=os.listdir(lp_path)
                                v_folders=natsort.natsorted(v_folders)
                                list_Na=[]
                                list_Nc=[]
                                list_err_Na=[]
                                list_err_Nc=[]
                                for vp in v_folders:
                                    v_path=lp_path+vp+'\\'
                                    t_files=os.listdir(v_path)
                                    t_files=natsort.natsorted(t_files)
                                    ### Creamos dos listas vacias donde guardar los valores de los contadores circulo y anillo
                                    Na=[]
                                    Nc=[]
                                    t_names=[]
                                    for t_file in t_files:
                                        if re.match(pleistoceno, t_file):
                                            t=re.split('[.]', t_file)[-1]
                                        else:
                                            t=re.split('[-_]', t_file)[-1]
                                        t_names.append(t)
                                        frame=trajectory()
                                        frame.read_frame(v_path+t_file)
                                        ContCirculo=0
                                        ContAnillo=0
                                        Lx=frame.Lx
                                        Ly=frame.Ly
                                        Np=frame.NumPart
                                        R1float=float(R1)
                                        gammafloat=float(gamma)
                                        R2=R1float-gammafloat
                                        ###cambiamos las coordenadas

                                        Xcorregido=frame.traj['x'].values-Lx/2
                                        Ycorregido=frame.traj['y'].values-Ly/2
                                        Rcorregido=numpy.sqrt(Xcorregido**2+Ycorregido**2)


                                        ###Contamos el numero de particulas atrapadas en el circulo interior

                                        ContCirculo=(Rcorregido<=R2).sum()

                                        ###Contamos el numero de particulas atrapadas en el anillo
                                        Rfuera=R2<Rcorregido
                                        Rdentro=Rcorregido<=R1float
                                        Ranillo=numpy.logical_and(Rfuera, Rdentro)

                                        ContAnillo=(Ranillo).sum()

                                        ### Guardamos los nuevos valores de los contadores al final de las listas
                                        Na.append(ContAnillo/Np)
                                        Nc.append(ContCirculo/Np)

                                    ### Vamos a crear las tablas para Na frente tiempo
                                    grafica_Na=pd.DataFrame({'t':t_names, 'Na':Na})
                                    grafica_Na.to_csv('E:\\1_TFG\\TFG\\Simulaciones\\Resultados-Tartarus-Punto\\'+Nf+'_'+lp+vp+'Na.csv', sep='\t', index=False)
                                    ### Vamos a crear las tablas para Nc frente tiempo
                                    grafica_Nc=pd.DataFrame({'t':t_names, 'Nc':Nc})
                                    grafica_Nc.to_csv('E:\\1_TFG\\TFG\\Simulaciones\\Resultados-Tartarus-Punto\\'+Nf+'_'+lp+vp+'Nc.csv', sep='\t', index=False)

                                    ### Seleccionamos los puntos que deberian encontrarse en el estado estacionario

                                    t_contador_Na=0
                                    obs_estacionarios_Na=[]
                                    for obs in Na:
                                        t_contador_Na += 1
                                        if t_contador_Na>Est_estacionario:
                                            obs_estacionarios_Na.append(obs)

                                    ### Repetimos lo mismo para Nc reseteando valores
                                    t_contador_Nc=0
                                    obs_estacionarios_Nc=[]
                                    for obs in Nc:
                                        t_contador_Nc += 1
                                        if t_contador_Nc>Est_estacionario:
                                            obs_estacionarios_Nc.append(obs)

                                    ### Calculamos los promedios de particulas atrapadas

                                    Na_promedio=(1/len(obs_estacionarios_Na))*sum(obs_estacionarios_Na)
                                    Nc_promedio=(1/len(obs_estacionarios_Nc))*sum(obs_estacionarios_Nc)

                                    error_Na=math.sqrt((1/len(obs_estacionarios_Na))*numpy.sum(numpy.square(obs_estacionarios_Na))-(Na_promedio)**2)
                                    error_Nc=math.sqrt((1/len(obs_estacionarios_Nc))*numpy.sum(numpy.square(obs_estacionarios_Nc))-(Nc_promedio)**2)

                                    list_Na.append(Na_promedio)
                                    list_Nc.append(Nc_promedio)
                                    list_err_Na.append(error_Na)
                                    list_err_Nc.append(error_Nc)
                                print(len(list_Na))
                                print(len(list_Nc))
                                print(len(list_err_Na))
                                print(len(list_err_Nc))
                                print(len(v_folders))

                                ### Vamos a crear las tablas para Nc frente v
                                grafica_Nc=pd.DataFrame({'v':v_folders, 'Na':list_Na, 'err_Na':list_err_Na, 'Nc':list_Nc, 'err_Nc':list_err_Nc})
                                grafica_Nc.to_csv('E:\\1_TFG\\TFG\\Simulaciones\\Resultados_promedios-Punto\\'+Nf+'_'+lp+' datos_finales.csv', sep='\t', index=False)

